Importation des bibliothéques

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [70]:
df = pd.read_csv('DataFrame_3G_After_Cleaning.csv')

Isolation Forest

In [71]:
import plotly.express as px
#import altair as alt
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go

In [72]:
x = df['PS Call Setup SR'].apply(lambda x: [x]).to_list()
iso_forest = IsolationForest(n_estimators=100,
                        max_samples="auto",
                        contamination=0.1,
                        random_state=42)
iso_forest.fit(x)
y_pred = iso_forest.predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]
#point that are 1 are outliers

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [73]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [74]:
iso_anomaly_df = pd.DataFrame(df)
iso_anomaly_df = iso_anomaly_df.loc[iso_anomaly_df['anomaly'] == 1]
iso_anomaly_df.head()


,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
7,7,23-12-21,0,86.34,76.22,10.12,157.31,99.27,0.71,94.63,2.06,100.00,99.54,96.89,0.0,672.96,1
23,23,01-08-22,0,76.68,67.55,9.14,123.39,98.87,3.00,94.32,3.23,97.75,99.42,96.14,40.0,628.69,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.0,490.41,1
25,25,01-10-22,0,64.63,57.21,7.42,110.09,98.29,3.87,89.49,3.79,97.88,99.60,95.49,100.0,670.35,1
27,27,01-12-22,0,83.56,74.45,9.11,133.89,99.29,3.97,94.22,2.86,97.93,99.76,96.63,100.0,823.60,1


In [75]:
fig = px.line(df,x='index', y='PS Call Setup SR', title='Unsupervised anomaly detection in PS Call Setup SR')
fig.add_trace(go.Scatter(x=iso_anomaly_df["index"].to_list(), y=iso_anomaly_df["PS Call Setup SR"].to_list(),
                            mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [76]:
nbr_val_inf_threshold = len(df[df['PS Call Setup SR'] < 95.00])
print("Nombres des valeurs inférieur à 95.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSSR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSSR = len(df[(df['anomaly'] == 1) & (df['PS Call Setup SR'] < 95.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSSR," parmi ",nbr_all_anomaly_PSSR, "anomalies détecter")

Nombres des valeurs inférieur à 95.00% sont : 16
Nombres des vrais anomalie détecter sont : 9  parmi  23 anomalies détecter


Local Outlier Factor

In [77]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=10, contamination=0.1)
y_pred = lof.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [78]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [79]:
lof_anomaly_df = pd.DataFrame(df)
lof_anomaly_df = lof_anomaly_df.loc[lof_anomaly_df['anomaly'] == 1]
lof_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
7,7,23-12-21,0,86.34,76.22,10.12,157.31,99.27,0.71,94.63,2.06,100.00,99.54,96.89,0.00,672.96,1
23,23,01-08-22,0,76.68,67.55,9.14,123.39,98.87,3.00,94.32,3.23,97.75,99.42,96.14,40.00,628.69,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.00,490.41,1
25,25,01-10-22,0,64.63,57.21,7.42,110.09,98.29,3.87,89.49,3.79,97.88,99.60,95.49,100.00,670.35,1
26,26,01-11-22,0,71.53,62.88,8.65,150.74,99.57,2.80,94.80,2.35,98.64,99.83,95.09,66.67,738.61,1


In [80]:

fig = px.line(df, x='index', y='PS Call Setup SR', title='Unsupervised anomaly detection in PS Call Setup SR')
fig.add_trace(go.Scatter(x=lof_anomaly_df["index"].to_list(), y=lof_anomaly_df["PS Call Setup SR"].to_list(),
                        mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [81]:
nbr_val_inf_threshold = len(df[df['PS Call Setup SR'] < 95.00])
print("Nombres des valeurs inférieur à 95.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSSR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSSR = len(df[(df['anomaly'] == 1) & (df['PS Call Setup SR'] < 95.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSSR," parmi ",nbr_all_anomaly_PSSR, "anomalies détecter")

Nombres des valeurs inférieur à 95.00% sont : 16
Nombres des vrais anomalie détecter sont : 11  parmi  22 anomalies détecter


OneClassSVM

In [82]:
from sklearn.svm import OneClassSVM
ocsvm = OneClassSVM(kernel='linear',
                     gamma='scale',
                     nu=0.1)
y_pred = ocsvm.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]


[0, 0, 1, 0, 0, 0, 1, 1, 0, 1]

In [83]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,1
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [84]:
ocsvm_anomaly_df = pd.DataFrame(df)
ocsvm_anomaly_df = ocsvm_anomaly_df.loc[ocsvm_anomaly_df['anomaly'] == 1]
ocsvm_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.00,99.57,96.43,0.00,755.79,1
6,6,22-12-21,0,89.15,78.33,10.82,142.73,99.54,0.68,94.94,1.76,100.00,99.85,96.54,0.00,815.16,1
7,7,23-12-21,0,86.34,76.22,10.12,157.31,99.27,0.71,94.63,2.06,100.00,99.54,96.89,0.00,672.96,1
9,9,25-12-21,0,60.16,52.90,7.26,118.15,98.97,2.93,95.07,2.83,98.01,99.52,95.97,66.67,645.25,1
23,23,01-08-22,0,76.68,67.55,9.14,123.39,98.87,3.00,94.32,3.23,97.75,99.42,96.14,40.00,628.69,1


In [85]:
fig = px.line(df, x='index', y='PS Call Setup SR', title='Unsupervise anomaly detection in PS Call Setup SR')
fig.add_trace(go.Scatter(x=ocsvm_anomaly_df["index"].to_list(),
                        y=ocsvm_anomaly_df["PS Call Setup SR"].to_list(),
                        mode='markers',
                        name='anomaly'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [86]:
nbr_val_inf_threshold = len(df[df['PS Call Setup SR'] < 95.00])
print("Nombres des valeurs inférieur à 95.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSSR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSSR = len(df[(df['anomaly'] == 1) & (df['PS Call Setup SR'] < 95.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSSR," parmi ",nbr_all_anomaly_PSSR, "anomalies détecter")

Nombres des valeurs inférieur à 95.00% sont : 16
Nombres des vrais anomalie détecter sont : 16  parmi  21 anomalies détecter
